## Import Dataset

folder structer
```
flower_photo/
    daisy/
    dandelion/
    roses/
    sunflowers/
    tulips/
```

In [ ]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

In [ ]:
import os
print("all folder names are : ", os.listdir('/root/.keras/datasets/flower_photos/'))
all_roses = list(data_dir.glob('roses/*'))
one_rose = all_roses[0]
print("one rose : ", one_rose)

all folder names are :  ['sunflowers', 'roses', 'daisy', 'dandelion', 'tulips', 'LICENSE.txt']
one rose :  /root/.keras/datasets/flower_photos/roses/15060816740_68e1b2c31b.jpg


In [ ]:
import PIL
PIL.Image.open(one_rose)

In [ ]:
image = PIL.Image.open(one_rose)
print(image.format)
print(image.size)
print(image.mode)

JPEG
(159, 240)
RGB


# Filter out corrupted images if any

# Dimensions Mean for random Image Set

In [ ]:
dim1 = []
dim2 = []
for one_rose in all_roses:
    image = PIL.Image.open(one_rose)
    d1,d2 = image.size
    dim1.append(d1)
    dim2.append(d2)

x_pix = int(np.mean(dim1) // 1)
y_pix = int(np.mean(dim2) // 1)
image_shape = (x_pix, y_pix, 3)

print(image_shape)

## Image Shape & Size

In [ ]:
batch_size = 32
input_shape = image_shape  # (x_pix, y_pix, 3)
image_size = (x_pix, y_pix)

# Generate Training & Test

>think like data_dir=flower_photos is present in your PWD

In [ ]:
"""
tf.keras.utils.image_dataset_from_directory

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)
   OR

tf.keras.preprocessing.image_dataset_from_directory
"""

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=image_size,
    batch_size=batch_size
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [ ]:
class_names = val_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


## Display images with class

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):  # taking 1st batch
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 364, 274, 3)
(32,)


## Data augmentation
 Data augmentation takes the approach of generating additional training data from your existing examples

https://www.tensorflow.org/tutorials/images/data_augmentation

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal", input_shape=image_shape),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
  ]
)

## Augmented images display

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

### AUTOTUNE

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Creating CNN

In [ ]:
model = models.Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=image_shape),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(len(class_names))
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

early_stop = EarlyStopping(monitor='val_loss', patience=10)
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
    callbacks=[early_stop]
)

model.summary()

In [ ]:
pd.DataFrame(model.history.history).plot()

# Using Image Data Generator

folder structer
```
flower_photo/
    train_data/
        daisy/
        dandelion/
        roses/
        sunflowers/
        tulips/
    test_data/
        daisy/
        dandelion/
        roses/
        sunflowers/
        tulips/
```

In [ ]:
image_gen = ImageDataGenerator(
    rescale=1/255,            # Rescale the image by normalzing it.
    rotation_range=20,        # rotate the image 20 degrees
    width_shift_range=0.10,   # Shift the pic width by a max of 5%
    height_shift_range=0.10,  # Shift the pic height by a max of 5%
    shear_range=0.1,          # Shear means cutting away part of the image (max 10%)
    zoom_range=0.1,           # Zoom in by 10% max
    horizontal_flip=True,     # Allo horizontal flipping
    vertical_flip=False,
    featurewise_center=True,
    featurewise_std_normalization=True,
    fill_mode='nearest'       # Fill in missing pixels with the nearest filled value
)

train_image_gen = image_gen.flow_from_directory(
    train_data,
    target_size=image_size,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size, shuffle=True,
    seed=None, save_to_dir=None, save_prefix='', save_format='png',
    follow_links=False, subset=None, interpolation='nearest'
)

test_image_gen = image_gen.flow_from_directory(
    test_data,
    target_size=image_size,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=batch_size, shuffle=False,
    seed=None, save_to_dir=None, save_prefix='', save_format='png',
    follow_links=False, subset=None, interpolation='nearest'
)

Found 3670 images belonging to 5 classes.
Found 3670 images belonging to 5 classes.


## Fit method for Image Generator

In [ ]:
"""
# it does not used now
model.fit_generator(
    train_image_gen,
    validation_data=test_image_gen,
    epochs=20,
    callbacks=[early_stop],
    use_multiprocessing=True
)
"""

model.fit(
    train_image_gen,
    validation_data=test_image_gen,
    steps_per_epoch=2000,
    epochs=50,
    validation_steps=800
)

model.evaluate_generator(test_image_gen)
pred_probabilities = model.predict_generator(test_image_gen)
predictions = pred_probabilities > 0.5
print(classification_report(test_image_gen.classes,predictions))

In [ ]:
pd.DataFrame(model.history.history).plot()

## Predict

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(sunflower_path, target_size=image_size)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)

score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)